In [ ]:
using BenchmarkTools
using Revise

In [ ]:
using Support

In [ ]:
SETTING = read_json("setting.json")

In [ ]:
using GCTGMT

In [ ]:
using FeatureSetEnrichment

In [ ]:
method_ = collect(
    keys(score_set_new(["a", "b"], [-1.0, 1.0], ["a"]; plot_process = false, plot = false)),
)

In [ ]:
benchmark_ = Vector{String}()

n_gene_ = Vector{Int64}()

benchmark_x_method_row_ = Vector{Vector{Float64}}()

In [ ]:
include("load_benchmark.jl")

using Normalization

In [ ]:
for benchmark_directory_path in read_directory(SETTING["benchmarks/"])

    element_, score_, set_to_element_, benchmark_set_ =
        load_benchmark(benchmark_directory_path)

    println(splitpath(benchmark_directory_path)[end])

    set_to_method_to_statistic = score_set_new(element_, score_, set_to_element_)

    set_ = collect(keys(set_to_method_to_statistic))

    set_x_method = convert_vector_of_vector_to_matrix([
        collect(values(set_to_method_to_statistic[set])) for set in set_
    ])

    for benchmark_set in benchmark_set_

        push!(
            benchmark_,
            string(splitpath(benchmark_directory_path)[end], '.', benchmark_set),
        )

        push!(n_gene_, length(set_to_element_[benchmark_set]))

        benchmark_x_method_row = Vector{Float64}()

        for method_i = 1:length(method_)

            set_score_ = set_x_method[:, method_i]

            is_negative_ = set_score_ .< 0

            is_positive_ = 0 .<= set_score_

            set_negative_ = set_[is_negative_]

            set_positive_ = set_[is_positive_]

            negative_ = .-normalize(.-set_score_[is_negative_], "1224") / sum(is_negative_)

            positive_ = normalize(set_score_[is_positive_], "1224") / sum(is_positive_)

            i_negative = findfirst(set_negative_ .== benchmark_set)

            i_positive = findfirst(set_positive_ .== benchmark_set)

            if i_negative != nothing

                benchmark_score = negative_[i_negative]

            elseif i_positive != nothing

                benchmark_score = positive_[i_positive]

            end

            push!(benchmark_x_method_row, benchmark_score)

        end

        push!(benchmark_x_method_row_, benchmark_x_method_row)

    end

end

In [ ]:
using CSV
using DataFrames

In [ ]:
df = DataFrame(convert_vector_of_vector_to_matrix(benchmark_x_method_row_), method_)

insertcols!(df, 1, "Benchmark" => benchmark_)

CSV.write(SETTING["benchmark_x_method.tsv"], df)